# Exercise 2b - PyOpenCL demo
The goal of this exercise is show the basic usecase of PyOpenCL. </br>
We will perform a simple elementwise operation on the GPU in two ways: 
- using PyOpenCL's array API
- manually by building and calling a C kernel.

In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array as cl_array

In PyopenCL we have to define our context manually. In the simplest case this means we select a platform, then a device, then initialize a context on this device.

In [2]:
platforms_list = cl.get_platforms()
platforms_list

[<pyopencl.Platform 'NVIDIA CUDA' at 0x1f9e680>]

In [3]:
devices_list = platforms_list[0].get_devices()
devices_list

[<pyopencl.Device 'Tesla T4' on 'NVIDIA CUDA' at 0x1c06750>]

In [35]:
# the total amount of GPU memory in bytes
devices_list[0].global_mem_size

15843721216

In [24]:
context = cl.Context(devices=devices_list)
context

<pyopencl.Context at 0x1d6b650 on <pyopencl.Device 'Tesla T4' on 'NVIDIA CUDA' at 0x1c06750>>

In [ ]:
queue = cl.CommandQueue(context, properties=cl.command_queue_properties.PROFILING_ENABLE)
queue

Initialize an input array on the CPU

In [ ]:
x_cpu = np.random.rand(int(1e6))

Transfer this array to the GPU

In [ ]:
x_gpu = cl_array.to_device(queue, x_cpu)

PyOpenCL arrays work like NumPy arrays

In [ ]:
print(type(x_cpu), type(x_gpu))

In [ ]:
x_gpu.shape

You can find the PyOpenCL equivalent of each NumPy math operation [here](https://documen.tician.de/pyopencl/array.html).

In [ ]:
import pyopencl.clmath as clmath

An elementwise operation

In [ ]:
y_gpu = 2 * clmath.sin(x_gpu) + clmath.exp(x_gpu)
print(type(y_gpu))

A reduction operation.

In [ ]:
z_cpu = np.sum(x_cpu)
z_gpu = cl_array.sum(x_gpu)
print(z_cpu, type(z_cpu), z_gpu, type(z_gpu))

Transfer data back to the CPU

In [ ]:
y_cpu = y_gpu.get()
z_cpu = z_gpu.get()
print(type(y_cpu), type(z_cpu))

We can do the same using manually defined low level C kernels

In [ ]:
source_str = r"""
__kernel
void elementwise(
    __global const double* x, 
    __global double* y)
{
    int i = get_global_id(0);
    y[i] = 2 * sin(x[i]) + exp(x[i]);
}
"""

Build and load the kernel function

In [ ]:
prg = cl.Program(context, source_str).build()
elementwise_kernel = prg.elementwise

Define an output array on the GPU

In [ ]:
y_gpu_2 = cl_array.zeros_like(x_gpu)

To call the kernel function we define the thread block size and the number of blocks.

In [ ]:
grid_size = len(x_cpu)
workgroup_size = 1

In [ ]:
elementwise_kernel(queue, (grid_size,), (workgroup_size,), x_gpu.data, y_gpu_2.data)

Check that the two outputs (using the API and the C kernel) are the same.

In [ ]:
y_cpu_2 = y_gpu_2.get()

In [ ]:
np.allclose(y_cpu, y_cpu_2)

#### Profiling

In [ ]:
queue = cl.CommandQueue(context, properties=cl.command_queue_properties.PROFILING_ENABLE)

In [ ]:
prg = cl.Program(context, source_str).build()
x_buffer = cl_array.to_device(queue, x_cpu)
y_buffer = cl_array.zeros_like(x_buffer)

In [ ]:
grid_size = len(x_cpu)
workgroup_size = 64

In [ ]:
event = prg.elementwise(queue, (grid_size,), (workgroup_size,), x_buffer.data, y_buffer.data)
event.wait()  # synchronize
elapsed = (event.profile.end - event.profile.start)*1e-3 # convert from [ns] to [us]
print(f"GPU kernel time: {int(elapsed)} us")